# Returns Analysis


In [1]:
# Install required packages
!pip install pandas numpy ipython


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Compute log-return diagnostics to understand drift, volatility, and autocorrelation before training transformers.
Tweak the configuration and re-run to focus on different subsets or export results.


In [2]:
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List

import numpy as np
import pandas as pd
from IPython.display import display


In [3]:
TRADING_DAYS_PER_YEAR = 252

@dataclass
class ReturnSummary:
    symbol: str
    category: str
    observations: int
    mean_log_return: float
    median_log_return: float
    std_log_return: float

    annual_drift: float
    annual_vol: float
    skew: float
    kurt: float
    pct_positive: float
    lag1_autocorr: float


def compute_log_returns(path: Path) -> ReturnSummary | None:
    try:
        df = pd.read_csv(path, parse_dates=['Date'])
    except Exception as exc:
        print(f'[WARN] Could not load {path}: {exc}')
        return None

    if df.empty or len(df) < 5:
        return None

    df = df.sort_values('Date')
    close = df['Close'].astype(float)
    log_returns = np.log(close).diff().dropna()
    if log_returns.empty:
        return None

    mean_lr = float(log_returns.mean())
    std_lr = float(log_returns.std(ddof=0))
    annual_vol = float(std_lr * np.sqrt(TRADING_DAYS_PER_YEAR))
    annual_drift = float(mean_lr * TRADING_DAYS_PER_YEAR)
    lag1 = float(log_returns.autocorr(lag=1) or 0.0)

    return ReturnSummary(
        symbol=path.stem,
        category=path.parent.name,
        observations=len(log_returns),
        mean_log_return=mean_lr,
        median_log_return=float(log_returns.median()),
        std_log_return=std_lr,
        annual_drift=annual_drift,
        annual_vol=annual_vol,
        skew=float(log_returns.skew()),
        kurt=float(log_returns.kurt()),
        pct_positive=float((log_returns > 0).mean()),
        lag1_autocorr=lag1,
    )


def gather_summaries(files: Iterable[Path]) -> List[ReturnSummary]:
    results: List[ReturnSummary] = []
    for path in files:
        summary = compute_log_returns(path)
        if summary is not None:
            results.append(summary)
    return results


def build_dataframe(summaries: Iterable[ReturnSummary]) -> pd.DataFrame:
    df = pd.DataFrame([s.__dict__ for s in summaries])
    if df.empty:
        raise RuntimeError('No return summaries generated. Check the data path.')
    df['signal_to_noise'] = df['mean_log_return'].abs() / df['std_log_return'].clip(lower=1e-12)
    df['sharpe_like'] = (
        df['mean_log_return'] * TRADING_DAYS_PER_YEAR
    ) / (df['std_log_return'] * np.sqrt(TRADING_DAYS_PER_YEAR)).replace(0, np.nan)
    return df


def print_global_highlights(df: pd.DataFrame) -> None:
    print('=== Return Distribution Highlights ===')
    print(f'Symbols analysed: {len(df)}')
    print(
        f'Median annual drift: {df['annual_drift'].median(): .4f} ' 
        f'(IQR: {df['annual_drift'].quantile(0.75): .4f} - {df['annual_drift'].quantile(0.25): .4f})'
    )
    print(
        f'Median annual volatility: {df['annual_vol'].median(): .2f} ' 
        f'(10th-90th pct: {df['annual_vol'].quantile(0.9): .2f} - {df['annual_vol'].quantile(0.1): .2f})'
    )
    median_autocorr = df['lag1_autocorr'].median()
    print(f'Median lag-1 autocorr: {median_autocorr: .4f}')

    by_category = (
        df.groupby('category')[['annual_vol', 'annual_drift', 'lag1_autocorr', 'signal_to_noise']]
        .median()
        .sort_index()
    )
    display(by_category)


def display_outliers(df: pd.DataFrame, metric: str, limit: int) -> None:
    if metric not in df:
        raise ValueError(f'Metric {metric} not found in dataframe')
    subset = df.nlargest(limit, metric)
    display(subset[['symbol', 'category', metric, 'annual_vol', 'annual_drift']])


## Configuration


In [4]:
DATA_ROOT = Path('../stock_data')
LIMIT = 5
METRIC = 'signal_to_noise'  # choose from: annual_vol, annual_drift, signal_to_noise, sharpe_like, lag1_autocorr
OUTPUT_CSV = None  # e.g. Path('outputs/returns_analysis.csv')


## Run Analysis


In [5]:
files = sorted(DATA_ROOT.glob('*/*.txt'))
if not files:
    raise RuntimeError(f'No symbol files found under {DATA_ROOT}')

summaries = gather_summaries(files)
df = build_dataframe(summaries)

print_global_highlights(df)
display_outliers(df, METRIC, LIMIT)

if OUTPUT_CSV:
    OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f'[INFO] Return stats saved to {OUTPUT_CSV}')


[WARN] Could not load ../stock_data/Stocks/accp.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/amrh.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/amrhw.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/asns.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/bbrx.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/bolt.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/boxl.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/bxg.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/ehr.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/fmax.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/gnst.us.txt: No columns to parse from file
[WARN] Could not load ../stock_data/Stocks/hayu.us.txt:

,annual_vol,annual_drift,lag1_autocorr,signal_to_noise
category,,,,
ETFs,0.208222,0.042106,-0.039641,0.024133
Stocks,0.384491,0.043734,-0.034313,0.018042


,symbol,category,signal_to_noise,annual_vol,annual_drift
3948,fxjp.us,Stocks,2.067565,0.108889,3.573907
7318,spro.us,Stocks,1.754141,0.308500,8.590543
4641,igz.us,Stocks,1.213097,0.032295,-0.621920
4058,ggzrw.us,Stocks,1.157038,1.106715,-20.327484
1617,alna.us,Stocks,1.051095,0.474434,7.916226
